# Spark Calculator

#### Purpose: 
To quickly pull out data from excel sheets and do the calculations. 

#### Instructions: 
1. Place files into local drives (OneDrive works as well). **Note**: Make sure there is only 1. 
2. Input Spark ID. 
3. Click `Cell` and `Run All`
4. Outputs file into "Spark Calc Output" folder as "SparkCalc_#ID_.xlsx"

In [36]:
# input name - File must be local or in GH One Drive
SparkID = "014_kk10"


In [37]:
# Other Parameters

volume = 40 # Total volume in wells of Spark Plate
dilutionFactor = 5 # put in 5 for 5X dilution, etc.


In [38]:
import pandas as pd
import os
import math

# find file within local directories
filepath = "/Users/"

for root, dirs, files in os.walk(filepath):
    for file in files:
        if (SparkID + ".xlsx") in file:
            sparkFile = (os.path.join(root, file))
            outpath = root

outDir = os.path.join(outpath, "Spark Calc Output")
if not os.path.exists(outDir):
    os.mkdir(outDir)
    
spark_data = pd.read_excel(sparkFile)

# function to pull out 260/320 abs and calculate concentrations
def colAbs(raw_spark, vol, dil):
    index260 = []
    index320 = []
    wells = []
    conc = []
    nmConc = []
    actualnM = []

    for names in raw_spark.iloc[35:(35 + 48), 0]:
        wells.append(names)
    for n260 in raw_spark.iloc[146:(146 + 48), 1]:
        index260.append(n260)
    for n320 in raw_spark.iloc[368:(368 + 48), 1]:
        index320.append(n320)
    # create one dataframe
    df = {"Wells": wells,
                 "abs260": index260,
                 "abs320": index320}
    df = pd.DataFrame(df)
    df = df.set_index("Wells")

    # Calculate concentration ng/uL ((260-320)*pi*4.69^2)/.002*4*vol)-8.5
    for i in range(len(df)):
        concalc = ((df.iloc[i, 0] - df.iloc[i, 1]) * math.pi * 4.69 ** 2 / (0.008 * vol)) - 8.5
        conc.append(round(concalc, 3))

    df.insert(2, "Concentration ng/uL", conc)

    # Calculate nM (Concentration/(660*65))*10^6
    for i in range(len(df)):
        nMole = ((df.iloc[i, 2])/(660*65))*10**6
        nmConc.append(round(nMole, 3))

    df.insert(3, "nM", nmConc)

    # Calculate "Actual nM in Plate"
    for i in range(len(df)):
        platenm = ((df.iloc[i, 3])*dil)
        actualnM.append(round(platenm, 3))
        
    df.insert(4, "Actual nM in Plate", actualnM)
    
    # Average nM
    df['Average nM'] = pd.Series(df.iloc[:len(df), 4].mean(), index=df.index[[0]])
    df['Average nM'] = df['Average nM'].fillna('')
    
    return df

dataFrame = colAbs(spark_data, volume, dilutionFactor)

wbName = "SparkCalc_" + SparkID + "_" + ".xlsx"
dataFrame.to_excel(os.path.join(outDir, wbName))

In [39]:
print(dataFrame)

       abs260  abs320  Concentration ng/uL       nM  Actual nM in Plate  \
Wells                                                                     
A1     0.1364  0.0442               11.410  265.967            1329.835   
B1     0.1439  0.0444               12.987  302.727            1513.635   
C1     0.1401  0.0445               12.144  283.077            1415.385   
D1     0.1422  0.0438               12.749  297.179            1485.895   
E1     0.1527  0.0557               12.447  290.140            1450.700   
F1     0.1410  0.0440               12.447  290.140            1450.700   
G1     0.1438  0.0444               12.965  302.214            1511.070   
H1     0.1483  0.0449               13.829  322.354            1611.770   
A2     0.1417  0.0447               12.447  290.140            1450.700   
B2     0.1450  0.0444               13.224  308.252            1541.260   
C2     0.1473  0.0490               12.728  296.690            1483.450   
D2     0.1561  0.0573    